# Lecture 10: End-to-End Machine Learning Pipeline - Programming Example

## Introduction: Building a Complete Production-Ready Forecasting System

Welcome to your final lecture! Over the past lectures, you've mastered essential machine learning (ML) skills: data cleaning, feature engineering, statistical analysis, visualization, linear regression, and ensemble methods. Today, you'll integrate all these skills into **one complete end-to-end pipeline** - taking Capital City Bikes from raw operational data through to production-ready demand forecasting models with rigorous evaluation and deployment recommendations.

Think of this as your graduation project: you'll work through the exact workflow professional ML engineers follow daily. Starting with data validation and ending with deployment decisions, you'll see how every skill you've learned fits into the bigger picture. This isn't just about building models - it's about building **trust in your predictions** through systematic data preparation, thoughtful feature engineering, rigorous evaluation, and transparent business communication.

Your client, Capital City Bikes, needs more than accurate forecasts - they need a complete system they can trust, explain to investors, and deploy confidently for their $2.3M municipal contract. Your end-to-end pipeline will demonstrate the professional rigor that justifies Series B funding rounds.

> **🚀 Interactive Learning Alert**
>
> This is a comprehensive hands-on capstone project integrating all course concepts. For the best experience:
>
> - **Click "Open in Colab"** at the bottom to run code interactively
> - **Execute each code cell** by pressing **Shift + Enter**
> - **Complete the challenges** to reinforce your end-to-end ML skills
> - **Think like a professional consultant** - every decision impacts deployment success

---

## Step 1: Data Quality Assessment and Cleaning

Before rushing into modeling, professional consultants always validate data quality. Even with "clean" datasets, we systematically verify integrity, check for edge cases, and ensure our foundation is solid. This disciplined approach prevents downstream surprises and builds stakeholder confidence.

**From the lecture "Data Quality & Cleaning Essentials"**, we learned the unified cleaning workflow: assess quality, identify issues, apply systematic fixes with transparency. Let's apply these principles to establish our reliable data foundation.

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Washington D.C. bike-sharing dataset
df = pd.read_csv("https://raw.githubusercontent.com/pmarcelino/predictive-modeling/main/datasets/dataset.csv")

print("=== STEP 1: DATA QUALITY ASSESSMENT ===\n")

# Convert datetime and sort chronologically for time series integrity
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values('datetime').reset_index(drop=True)

# Quick data quality checks
print("--- Structural Overview ---")
print(f"Dataset shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Time range: {df['datetime'].min()} to {df['datetime'].max()}")
print(f"Total hours: {len(df):,}\n")

# Check for missing values
print("--- Missing Data Assessment ---")
missing_summary = df.isnull().sum()
missing_pct = (missing_summary / len(df)) * 100
missing_df = pd.DataFrame({'Missing_Count': missing_summary, 'Missing_%': missing_pct})
print(missing_df[missing_df['Missing_Count'] > 0])

if missing_df['Missing_Count'].sum() == 0:
    print("✓ No missing values detected - excellent data quality!\n")
else:
    print(f"⚠ Found missing values - will handle in cleaning step\n")

# Value range validation
print("--- Value Range Validation ---")
reasonable_ranges = {
    'temp': (-20, 45),
    'humidity': (0, 100),
    'windspeed': (0, 50),
    'count': (0, 1000)
}

issues_found = False
for column, (min_val, max_val) in reasonable_ranges.items():
    if column in df.columns:
        below = (df[column] < min_val).sum()
        above = (df[column] > max_val).sum()
        
        if below > 0 or above > 0:
            issues_found = True
            print(f"⚠ {column}: {below} values too low, {above} values too high")
        else:
            print(f"✓ {column}: All values within expected range [{min_val}, {max_val}]")

if not issues_found:
    print("\n✓ All values within reasonable ranges - data validated!")

print(f"\n--- Timeline Continuity Check ---")
# Check for duplicate timestamps
duplicates = df['datetime'].duplicated().sum()
print(f"Duplicate timestamps: {duplicates}")

# Check for missing hours
time_min, time_max = df['datetime'].min(), df['datetime'].max()
expected_hours = pd.date_range(time_min, time_max, freq='h')
actual_hours = df['datetime'].unique()
missing_hours = len(expected_hours) - len(actual_hours)
print(f"Missing hours in timeline: {missing_hours}")

if duplicates == 0 and missing_hours == 0:
    print("✓ Perfect timeline continuity - ready for time series analysis!\n")

# Create clean dataset for subsequent analysis
print("--- Creating Clean Dataset ---")
df_clean = df.copy()
print(f"✓ df_clean created: {df_clean.shape[0]} rows × {df_clean.shape[1]} columns")
print("✓ Data quality validated and ready for feature engineering!")

**What this does:**
- Loads dataset and validates data quality systematically
- Checks for missing values, impossible values, timeline continuity
- Creates `df_clean` as validated foundation for the next steps

**Business value:**
This systematic validation ensures Capital City Bikes can trust their data foundation. Any quality issues are identified and resolved transparently before they contaminate modeling results.

### Challenge 1: Outlier Detection and Investigation

Your client asks: "Are there any unusual windspeed conditions we should investigate before modeling?" Implement outlier detection using the IQR method to identify extreme windspeed periods.

In [ ]:
# Your code here - detect outliers using IQR method

# Calculate IQR (Interquartile Range) for windspeed
Q1 = df_clean['_____'].quantile(_____)  # 25th percentile
Q3 = df_clean['_____'].quantile(_____)  # 75th percentile
IQR = _____ - _____

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = df_clean[(df_clean['windspeed'] < _____) | (df_clean['windspeed'] > _____)]

print(f"=== OUTLIER DETECTION (IQR METHOD) ===")
print(f"Q1 (25th percentile): {Q1:.2f} (scaled)")
print(f"Q3 (75th percentile): {Q3:.2f} (scaled)")
print(f"IQR: {IQR:.2f} (scaled)")
print(f"Lower bound: {lower_bound:.2f} (scaled)")
print(f"Upper bound: {upper_bound:.2f} (scaled)")
print(f"Outliers detected: {len(outliers)} observations ({len(outliers)/len(df_clean)*100:.1f}%)")

# Examine top outliers
if len(outliers) > 0:
    print("\nTop 5 high-windspeed outliers:")
    print(outliers.nlargest(5, 'windspeed')[['datetime', 'temp', 'windspeed', 'weather', 'count']])

<details>
<summary>💡 <strong>Tip</strong> (click to expand)</summary>

The IQR method uses quartiles to define outlier boundaries: Q1 (25th percentile) and Q3 (75th percentile) define the "box" in a box plot, IQR = Q3 - Q1 is the box height. Standard outlier boundaries are Q1 - 1.5×IQR (lower) and Q3 + 1.5×IQR (upper). Use `.quantile(0.25)` for Q1 and `.quantile(0.75)` for Q3. Filter outliers with boolean indexing: `df[(df['windspeed'] < lower_bound) | (df['windspeed'] > upper_bound)]`. The `|` operator means "OR" - values below lower bound OR above upper bound. Examine outliers using `.nlargest(5, 'windspeed')` to see if they're legitimate events (storms, extreme weather) or measurement errors. At this stage, windspeed is still in original units (0-67 normalized scale from the dataset), so outlier boundaries will be in these original units. High windspeed outliers may indicate extreme weather events that significantly impact bike demand, so we typically flag them for transparency in reporting to clients.
</details>

<details>
<summary>🤫 <strong>Solution</strong> (click to expand)</summary>

```python
# Calculate IQR for windspeed
Q1 = df_clean['windspeed'].quantile(0.25)
Q3 = df_clean['windspeed'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = df_clean[(df_clean['windspeed'] < lower_bound) | (df_clean['windspeed'] > upper_bound)]

print(f"=== OUTLIER DETECTION (IQR METHOD) ===")
print(f"Q1 (25th percentile): {Q1:.2f} (scaled)")
print(f"Q3 (75th percentile): {Q3:.2f} (scaled)")
print(f"IQR: {IQR:.2f} (scaled)")
print(f"Lower bound: {lower_bound:.2f} (scaled)")
print(f"Upper bound: {upper_bound:.2f} (scaled)")
print(f"Outliers detected: {len(outliers)} observations ({len(outliers)/len(df_clean)*100:.1f}%)")

# Examine top outliers
if len(outliers) > 0:
    print("\nTop 5 high-windspeed outliers:")
    print(outliers.nlargest(5, 'windspeed')[['datetime', 'temp', 'windspeed', 'weather', 'count']])
    
    # Add outlier flag for transparency
    df_clean['is_windspeed_outlier'] = ((df_clean['windspeed'] < lower_bound) | (df_clean['windspeed'] > upper_bound)).astype(int)
    print(f"\n✓ Windspeed outliers flagged for transparency in analysis")
```
</details>

---

## Step 2: Feature Engineering and Preprocessing

With validated data, we now engineer features that expose patterns for machine learning. **From the lectures in the module "Data Preparation"**, we learned categorical encoding, feature scaling, cyclical time encoding, and temporal features. These transformations convert raw data into optimized machine learning inputs.

In [ ]:
# Import preprocessing tools
from sklearn.preprocessing import StandardScaler

print("=== STEP 2: FEATURE ENGINEERING ===\n")

# Extract temporal components from datetime
print("--- Temporal Feature Extraction ---")
df_clean['hour'] = df_clean['datetime'].dt.hour
df_clean['dayofweek'] = df_clean['datetime'].dt.dayofweek
df_clean['month'] = df_clean['datetime'].dt.month

print(f"✓ Extracted: hour (0-23), dayofweek (0-6), month (1-12)")

# Binary indicators for business-critical conditions
print("\n--- Binary Indicator Creation ---")
rush_hours = [7, 8, 9, 17, 18, 19]
df_clean['is_rush_hour'] = df_clean['hour'].isin(rush_hours).astype(int)
df_clean['is_weekend'] = df_clean['dayofweek'].isin([5, 6]).astype(int)
df_clean['is_night'] = ((df_clean['hour'] >= 22) | (df_clean['hour'] <= 5)).astype(int)

print(f"✓ is_rush_hour: {df_clean['is_rush_hour'].mean()*100:.1f}% of hours")
print(f"✓ is_weekend: {df_clean['is_weekend'].mean()*100:.1f}% of hours")
print(f"✓ is_night: {df_clean['is_night'].mean()*100:.1f}% of hours")

# Cyclical encoding for continuous time (hour and day cycles)
print("\n--- Cyclical Time Encoding ---")
df_clean['hour_sin'] = np.sin(2 * np.pi * df_clean['hour'] / 24)
df_clean['hour_cos'] = np.cos(2 * np.pi * df_clean['hour'] / 24)
df_clean['day_sin'] = np.sin(2 * np.pi * df_clean['dayofweek'] / 7)
df_clean['day_cos'] = np.cos(2 * np.pi * df_clean['dayofweek'] / 7)

print(f"✓ Hour cycle: hour_sin, hour_cos (24-hour continuity)")
print(f"✓ Day cycle: day_sin, day_cos (7-day continuity)")

# Verify cyclical encoding works (hour 23 should be close to hour 0)
hour_23_encoding = df_clean[df_clean['hour'] == 23][['hour_sin', 'hour_cos']].iloc[0]
hour_0_encoding = df_clean[df_clean['hour'] == 0][['hour_sin', 'hour_cos']].iloc[0]
print(f"\nVerification - Hour 23: sin={hour_23_encoding['hour_sin']:.3f}, cos={hour_23_encoding['hour_cos']:.3f}")
print(f"Verification - Hour 0:  sin={hour_0_encoding['hour_sin']:.3f}, cos={hour_0_encoding['hour_cos']:.3f}")
print("✓ Values are close - cyclical continuity preserved!")

# Feature scaling for weather variables
print("\n--- Feature Scaling (Weather Variables) ---")
weather_features = ['temp', 'atemp', 'humidity', 'windspeed']

print("Before scaling:")
print(df_clean[weather_features].describe().loc[['mean', 'std']].round(2))

scaler = StandardScaler()
df_clean[weather_features] = scaler.fit_transform(df_clean[weather_features])

print("\nAfter scaling:")
print(df_clean[weather_features].describe().loc[['mean', 'std']].round(3))
print("✓ Weather features standardized (mean≈0, std≈1)")

# Lag features for sequential patterns
print("\n--- Lag Feature Creation ---")
df_clean['demand_lag_1h'] = df_clean['count'].shift(1)
df_clean['demand_lag_24h'] = df_clean['count'].shift(24)

# Handle NaN values created by shift
df_clean['demand_lag_1h'] = df_clean['demand_lag_1h'].fillna(df_clean['count'].mean())
df_clean['demand_lag_24h'] = df_clean['demand_lag_24h'].fillna(df_clean['count'].mean())

print(f"✓ demand_lag_1h: Previous hour's demand (captures momentum)")
print(f"✓ demand_lag_24h: Same hour yesterday (captures daily patterns)")

# Check lag feature correlations
corr_1h = df_clean['count'].corr(df_clean['demand_lag_1h'])
corr_24h = df_clean['count'].corr(df_clean['demand_lag_24h'])
print(f"\nLag correlations with current demand:")
print(f"  1-hour lag:  r = {corr_1h:.3f}")
print(f"  24-hour lag: r = {corr_24h:.3f}")

print(f"\n✓ Feature engineering complete: {len(df_clean.columns)} total columns")
print("✓ Ready for exploratory analysis and modeling!")

**What this does:**
- Extracts temporal components (hour, day, month) from datetime
- Creates binary indicators for operational segments (rush hour, weekend, night)
- Implements cyclical encoding ensuring time continuity (11pm → midnight)
- Scales weather features to comparable ranges (mean=0, std=1)
- Generates lag features capturing sequential demand patterns
- Validates feature quality through correlation analysis

**Business value:**
These engineered features transform raw measurements into ML-ready predictors that expose temporal patterns, operational contexts, and sequential dependencies - the patterns that drive accurate demand forecasting.

### Challenge 2: Create Rolling Window Features

Your client asks: "Can we capture demand trends over recent hours? I'd like to understand short-term and daily patterns." Implement rolling window features that summarize recent demand history.

In [ ]:
# Your code here - create rolling window features

# 3-hour rolling average (short-term trend)
df_clean['demand_3h_avg'] = df_clean['count'].shift(1).rolling(window=_____, min_periods=1)._____()

# 24-hour rolling average (daily baseline)
df_clean['demand_24h_avg'] = df_clean['count'].shift(1).rolling(window=_____, min_periods=1)._____()

print("=== ROLLING WINDOW FEATURES ===")
print(f"✓ demand_3h_avg: Rolling average of past 3 hours (short-term trend)")
print(f"✓ demand_24h_avg: Rolling average of past 24 hours (daily baseline)")

# Visualize rolling patterns
print(f"\n3-hour rolling average statistics:")
print(f"  Mean: {df_clean['demand_3h_avg'].mean():.2f}")
print(f"  Std: {df_clean['demand_3h_avg'].std():.2f}")

print(f"\n24-hour rolling average statistics:")
print(f"  Mean: {df_clean['demand_24h_avg'].mean():.2f}")
print(f"  Std: {df_clean['demand_24h_avg'].std():.2f}")

<details>
<summary>💡 <strong>Tip</strong> (click to expand)</summary>

Rolling window features use pandas `.rolling(window=n)` to calculate statistics over the past n observations. ALWAYS use `.shift(1)` before `.rolling()` to prevent data leakage - without it, the current value you're trying to predict gets included in the "past" window! The pattern is: `df['count'].shift(1).rolling(window=3).mean()` for a 3-hour average. Use `window=3` for short-term trends and `window=24` for daily baselines. Set `min_periods=1` to handle the first few rows where full windows aren't available. The 3-hour average captures immediate trends (is demand rising or falling in the past few hours?), while the 24-hour average provides a stable daily baseline that accounts for typical patterns at each hour. These features help models understand not just current conditions but recent demand trajectories. For example, if demand_3h_avg is significantly higher than demand_24h_avg, it signals an unusual surge requiring proactive bike rebalancing.
</details>

<details>
<summary>🤫 <strong>Solution</strong> (click to expand)</summary>

```python
# 3-hour rolling average (short-term trend)
df_clean['demand_3h_avg'] = df_clean['count'].shift(1).rolling(window=3, min_periods=1).mean()

# 24-hour rolling average (daily baseline)
df_clean['demand_24h_avg'] = df_clean['count'].shift(1).rolling(window=24, min_periods=1).mean()

print("=== ROLLING WINDOW FEATURES ===")
print(f"✓ demand_3h_avg: Rolling average of past 3 hours (short-term trend)")
print(f"✓ demand_24h_avg: Rolling average of past 24 hours (daily baseline)")

# Visualize rolling patterns
print(f"\n3-hour rolling average statistics:")
print(f"  Mean: {df_clean['demand_3h_avg'].mean():.2f}")
print(f"  Std: {df_clean['demand_3h_avg'].std():.2f}")

print(f"\n24-hour rolling average statistics:")
print(f"  Mean: {df_clean['demand_24h_avg'].mean():.2f}")
print(f"  Std: {df_clean['demand_24h_avg'].std():.2f}")

# Show correlation with current demand
corr_3h = df_clean['count'].corr(df_clean['demand_3h_avg'])
corr_24h = df_clean['count'].corr(df_clean['demand_24h_avg'])
print(f"\nCorrelation with current demand:")
print(f"  3-hour rolling avg:  r = {corr_3h:.3f}")
print(f"  24-hour rolling avg: r = {corr_24h:.3f}")
```
</details>

---

## Step 3: Exploratory Data Analysis and Visualization

Before modeling, we explore our engineered features to validate they capture real patterns. **From the lectures in module "Data Exploration"**, we learned statistical analysis and visualization techniques. This exploration builds confidence in our feature engineering and reveals insights that guide modeling decisions.

In [ ]:
print("=== STEP 3: EXPLORATORY DATA ANALYSIS ===\n")

# Descriptive statistics for key variables
print("--- Descriptive Statistics ---")
key_metrics = ['temp', 'humidity', 'windspeed', 'count']
stats_summary = df_clean[key_metrics].describe().round(2)
print(stats_summary)

# Seasonal demand comparison
print("\n--- Seasonal Demand Patterns ---")
season_names = {1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'}
df_clean['season_name'] = df_clean['season'].map(season_names)

seasonal_stats = df_clean.groupby('season_name')['count'].agg(['mean', 'std', 'count']).round(1)
print(seasonal_stats)

peak_season = seasonal_stats['mean'].idxmax()
low_season = seasonal_stats['mean'].idxmin()
print(f"\nPeak season: {peak_season} ({seasonal_stats.loc[peak_season, 'mean']:.0f} bikes/hour)")
print(f"Low season: {low_season} ({seasonal_stats.loc[low_season, 'mean']:.0f} bikes/hour)")

# Correlation analysis with engineered features
print("\n--- Correlation Analysis ---")
correlation_features = ['temp', 'humidity', 'hour', 'dayofweek', 'month',
                        'is_rush_hour', 'is_weekend', 'demand_lag_1h', 'demand_lag_24h', 'count']

# Calculate correlations with target variable
correlations = df_clean[correlation_features].corr()['count'].sort_values(ascending=False)
print("\nFeatures ranked by correlation with demand:")
print(correlations.round(3))

print(f"\nStrongest positive predictor: {correlations.index[1]} (r = {correlations.iloc[1]:.3f})")
print(f"Strongest negative predictor: {correlations[correlations < 0].index[0]} (r = {correlations[correlations < 0].iloc[0]:.3f})")

# Create visualizations showcasing engineered features
print("\n--- Creating Visualizations ---")

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Panel 1: Lag Feature Autocorrelation - 1-hour lag
from scipy import stats
slope_1h, intercept_1h, r_1h, _, _ = stats.linregress(df_clean['demand_lag_1h'], df_clean['count'])
axes[0, 0].scatter(df_clean['demand_lag_1h'], df_clean['count'], 
                   alpha=0.3, s=10, color='steelblue', edgecolors='none')
axes[0, 0].plot(df_clean['demand_lag_1h'], 
                slope_1h * df_clean['demand_lag_1h'] + intercept_1h,
                'r--', linewidth=2.5, label=f'Linear fit (r = {r_1h:.3f})')
axes[0, 0].set_xlabel('Previous Hour Demand (bikes)', fontsize=11)
axes[0, 0].set_ylabel('Current Hour Demand (bikes)', fontsize=11)
axes[0, 0].set_title('1-Hour Lag Correlation (Momentum)', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Panel 2: Lag Feature Autocorrelation - 24-hour lag
slope_24h, intercept_24h, r_24h, _, _ = stats.linregress(df_clean['demand_lag_24h'], df_clean['count'])
axes[0, 1].scatter(df_clean['demand_lag_24h'], df_clean['count'], 
                   alpha=0.3, s=10, color='darkgreen', edgecolors='none')
axes[0, 1].plot(df_clean['demand_lag_24h'], 
                slope_24h * df_clean['demand_lag_24h'] + intercept_24h,
                'r--', linewidth=2.5, label=f'Linear fit (r = {r_24h:.3f})')
axes[0, 1].set_xlabel('Same Hour Yesterday Demand (bikes)', fontsize=11)
axes[0, 1].set_ylabel('Current Hour Demand (bikes)', fontsize=11)
axes[0, 1].set_title('24-Hour Lag Correlation (Daily Pattern)', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Panel 3: Binary Feature Impact - Combined view
binary_features_data = [
    ('Rush\nHour', df_clean[df_clean['is_rush_hour'] == 0]['count'], 
     df_clean[df_clean['is_rush_hour'] == 1]['count']),
    ('Weekend', df_clean[df_clean['is_weekend'] == 0]['count'], 
     df_clean[df_clean['is_weekend'] == 1]['count']),
    ('Night\nTime', df_clean[df_clean['is_night'] == 0]['count'], 
     df_clean[df_clean['is_night'] == 1]['count'])
]

positions = [1, 2, 4, 5, 7, 8]
colors = ['lightblue', 'lightcoral', 'lightblue', 'lightcoral', 'lightblue', 'lightcoral']
data_to_plot = []
for _, no, yes in binary_features_data:
    data_to_plot.extend([no, yes])

bp = axes[1, 0].boxplot(data_to_plot, positions=positions, widths=0.6,
                        patch_artist=True, showmeans=True,
                        boxprops=dict(facecolor='white', edgecolor='black'),
                        medianprops=dict(color='red', linewidth=2),
                        meanprops=dict(marker='D', markerfacecolor='green', markersize=6))

# Color the boxes
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

axes[1, 0].set_xticks([1.5, 4.5, 7.5])
axes[1, 0].set_xticklabels([name for name, _, _ in binary_features_data])
axes[1, 0].set_ylabel('Hourly Demand (bikes)', fontsize=11)
axes[1, 0].set_title('Binary Feature Impact on Demand Distribution', fontsize=12, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)
axes[1, 0].legend([bp['boxes'][0], bp['boxes'][1]], ['No (0)', 'Yes (1)'], 
                  loc='upper right', fontsize=9)

# Panel 4: Correlation Heatmap of Engineered Features
heatmap_features = ['temp', 'humidity', 'windspeed', 'hour_sin', 'hour_cos', 
                    'is_rush_hour', 'is_weekend', 'is_night', 
                    'demand_lag_1h', 'demand_lag_24h', 'count']
corr_matrix = df_clean[heatmap_features].corr()

im = axes[1, 1].imshow(corr_matrix, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
axes[1, 1].set_xticks(range(len(heatmap_features)))
axes[1, 1].set_yticks(range(len(heatmap_features)))
axes[1, 1].set_xticklabels(heatmap_features, rotation=45, ha='right', fontsize=9)
axes[1, 1].set_yticklabels(heatmap_features, fontsize=9)
axes[1, 1].set_title('Feature Correlation Matrix', fontsize=12, fontweight='bold')

# Add correlation values for top/bottom relationships with target
for i in range(len(heatmap_features)):
    for j in range(len(heatmap_features)):
        if abs(corr_matrix.iloc[i, j]) > 0.5 or (i == len(heatmap_features)-1) or (j == len(heatmap_features)-1):
            text = axes[1, 1].text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                                  ha='center', va='center', fontsize=8,
                                  color='white' if abs(corr_matrix.iloc[i, j]) > 0.5 else 'black')

# Add colorbar
cbar = plt.colorbar(im, ax=axes[1, 1], fraction=0.046, pad=0.04)
cbar.set_label('Correlation', fontsize=10)

plt.tight_layout()
plt.show()

print("\n✓ Visualizations complete - engineered features validated!")
print(f"✓ Lag features show strong predictive power: 1h (r={r_1h:.3f}), 24h (r={r_24h:.3f})")
print(f"✓ Binary features create meaningful demand segmentation")
print(f"✓ Feature correlation matrix reveals relationships and independence")

**What this does:**
- Creates four visualization panels
- **Lag correlation scatters** (panels 1-2): Validate that demand_lag_1h and demand_lag_24h strongly predict current demand (r ≈ 0.8-0.9), proving sequential patterns are captured
- **Binary feature box plots** (panel 3): Show how is_rush_hour, is_weekend, and is_night create distinct demand distributions - visual evidence these flags segment operational contexts effectively
- **Correlation heatmap** (panel 4): Reveals relationships among ALL engineered features - identifies redundant features (high inter-feature correlation) vs. complementary features (low inter-feature correlation but high correlation with target)

**Business value:**
These visualizations prove that Step 2 feature engineering works BEFORE investing in modeling. Strong lag correlations justify real-time demand monitoring systems. Binary feature segmentation guides operational policy (different strategies for rush hour vs. night). Correlation matrix prevents redundant data collection and identifies most cost-effective monitoring investments.

### Challenge 3: Create Diverse Visualization Types

Your client requests: "Can you create additional visualizations showing demand distributions and multi-dimensional patterns? We want to understand variability and combined effects." Create box plots and heatmaps to reveal distribution patterns and interaction effects.

In [ ]:
# Your code here - create box plots and heatmap

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Panel 1: Box plot - demand distribution by weather condition
weather_labels = {1: 'Clear', 2: 'Mist', 3: 'Light Rain', 4: 'Heavy Rain'}
df_clean['weather_label'] = df_clean['weather'].map(weather_labels)

# Create box plot showing distribution for each weather type
weather_data = [df_clean[df_clean['weather'] == i]['count'] for i in [_____, _____, _____, _____]]
axes[0].boxplot(weather_data, tick_labels=_____)
axes[0].set_xlabel('Weather Condition', fontsize=11)
axes[0].set_ylabel('Hourly Demand (bikes)', fontsize=11)
axes[0].set_title('Demand Distribution by Weather', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Panel 2: Heatmap - demand by hour and day of week
hourly_daily = df_clean.pivot_table(values='_____', index='_____', columns='_____', aggfunc='mean')
sns.heatmap(hourly_daily, cmap='YlOrRd', ax=axes[1], cbar_kws={'label': 'Avg Demand'})
axes[1].set_xlabel('Day of Week (0=Mon, 6=Sun)', fontsize=11)
axes[1].set_ylabel('Hour of Day', fontsize=11)
axes[1].set_title('Demand Heatmap: Hour × Day Patterns', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("=== ADVANCED VISUALIZATION INSIGHTS ===")
print("Box plot: Shows demand variability across weather conditions")
print("Heatmap: Reveals hour-day interaction patterns (weekday rush hours vs weekend leisure)")

<details>
<summary>💡 <strong>Tip</strong> (click to expand)</summary>

Box plots show distribution characteristics: the box spans Q1 to Q3 (middle 50%), the line inside is the median, whiskers extend to typical min/max, and dots are outliers. Create data list using list comprehension: `[df_clean[df_clean['weather'] == i]['count'] for i in [1, 2, 3, 4]]` gives you four arrays (one per weather type). Pass this list to `plt.boxplot()` with `labels=['Clear', 'Mist', 'Light Rain', 'Heavy Rain']`. For heatmaps, use `df.pivot_table(values='count', index='hour', columns='dayofweek', aggfunc='mean')` to create a matrix where rows=hours, columns=days, cells=average demand. Then `sns.heatmap()` with `cmap='YlOrRd'` creates color-coded visualization where darker colors = higher demand. The heatmap will instantly reveal weekday morning/evening peaks versus weekend midday patterns - visual confirmation of temporal feature importance!
</details>

<details>
<summary>🤫 <strong>Solution</strong> (click to expand)</summary>

```python
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Panel 1: Box plot - demand distribution by weather condition
weather_labels = {1: 'Clear', 2: 'Mist', 3: 'Light Rain', 4: 'Heavy Rain'}
df_clean['weather_label'] = df_clean['weather'].map(weather_labels)

weather_data = [df_clean[df_clean['weather'] == i]['count'] for i in [1, 2, 3, 4]]
axes[0].boxplot(weather_data, tick_labels=['Clear', 'Mist', 'Light Rain', 'Heavy Rain'])
axes[0].set_xlabel('Weather Condition', fontsize=11)
axes[0].set_ylabel('Hourly Demand (bikes)', fontsize=11)
axes[0].set_title('Demand Distribution by Weather', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Panel 2: Heatmap - demand by hour and day of week
hourly_daily = df_clean.pivot_table(values='count', index='hour', columns='dayofweek', aggfunc='mean')
sns.heatmap(hourly_daily, cmap='YlOrRd', ax=axes[1], cbar_kws={'label': 'Avg Demand'}, fmt='.0f')
axes[1].set_xlabel('Day of Week (0=Mon, 6=Sun)', fontsize=11)
axes[1].set_ylabel('Hour of Day', fontsize=11)
axes[1].set_title('Demand Heatmap: Hour × Day Patterns', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("=== ADVANCED VISUALIZATION INSIGHTS ===")
print("✓ Box plot reveals: Clear weather has highest median and widest range")
print("✓ Heatmap shows: Weekday hours 8 & 17 are hottest (rush hours)")
print("✓ Pattern validated: Temporal features critical for accurate forecasting")
```
</details>

---

## Step 4: Linear Regression Baseline Model

With validated features, we build our baseline model. **From the lecture "Linear Models for Prediction"**, Linear Regression provides interpretable predictions with clear coefficient interpretation. This baseline establishes performance that advanced models must beat to justify their complexity.

In [ ]:
# Import modeling tools
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

print("=== STEP 4: LINEAR REGRESSION BASELINE ===\n")

# Define feature columns for modeling
feature_columns = [
    # Weather features (scaled)
    'temp', 'atemp', 'humidity', 'windspeed',
    # Temporal features (both raw and cyclical)
    'hour', 'hour_sin', 'hour_cos', 'dayofweek', 'day_sin', 'day_cos', 'month', 'season',
    # Categorical
    'weather',
    # Binary indicators
    'workingday', 'holiday', 'is_rush_hour', 'is_weekend', 'is_night',
    # Lag features
    'demand_lag_1h', 'demand_lag_24h'
]

X = df_clean[feature_columns]
y = df_clean['count']

print(f"Feature matrix: {X.shape[0]} observations × {X.shape[1]} features")
print(f"Features: {', '.join(feature_columns[:5])}... (and {len(feature_columns)-5} more)")
print(f"Target: count (hourly bike rentals)\n")

# Chronological train-test split (80/20)
print("--- Chronological Train-Test Split ---")
split_index = int(len(df_clean) * 0.8)

X_train = X.iloc[:split_index]
X_test = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]

train_period = f"{df_clean.iloc[:split_index]['datetime'].min()} to {df_clean.iloc[:split_index]['datetime'].max()}"
test_period = f"{df_clean.iloc[split_index:]['datetime'].min()} to {df_clean.iloc[split_index:]['datetime'].max()}"

print(f"Training set: {len(X_train):,} observations ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Period: {train_period}")
print(f"Testing set:  {len(X_test):,} observations ({len(X_test)/len(X)*100:.1f}%)")
print(f"  Period: {test_period}")
print("✓ Chronological order preserved - training on past, testing on future\n")

# Train Linear Regression model
print("--- Training Linear Regression ---")
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

print("✓ Linear model trained successfully!")
print(f"Coefficients learned: {len(linear_model.coef_)} parameters")

# Show top 3 most influential features
feature_importance_linear = pd.DataFrame({
    'feature': feature_columns,
    'coefficient': linear_model.coef_
})
feature_importance_linear['abs_coef'] = np.abs(feature_importance_linear['coefficient'])
top_features = feature_importance_linear.nlargest(3, 'abs_coef')

print("\nTop 3 most influential features:")
for idx, row in top_features.iterrows():
    print(f"  {row['feature']}: {row['coefficient']:+.2f} (larger magnitude = stronger impact)")

# Evaluate on test set
print("\n--- Model Evaluation ---")
test_predictions_linear = linear_model.predict(X_test)
test_r2_linear = r2_score(y_test, test_predictions_linear)

print(f"Test R²: {test_r2_linear:.4f} ({test_r2_linear*100:.2f}% variance explained)")

if test_r2_linear > 0.80:
    print("✓ Excellent performance - model explains >80% of demand variation")
elif test_r2_linear > 0.60:
    print("✓ Good performance - model captures major demand patterns")
elif test_r2_linear > 0.40:
    print("~ Moderate performance - room for improvement with advanced models")
else:
    print("⚠ Limited performance - advanced models likely needed")

# Show example predictions
print("\n--- Example Predictions (First 5 Test Observations) ---")
comparison = pd.DataFrame({
    'Actual': y_test.iloc[:5].values,
    'Predicted': test_predictions_linear[:5],
    'Error': y_test.iloc[:5].values - test_predictions_linear[:5]
})
print(comparison.round(1))

print(f"\n✓ Linear Regression baseline established: R² = {test_r2_linear:.4f}")

**What this does:**
- Selects 17 engineered features for modeling (weather, temporal, binary, lags)
- Creates chronological 80/20 split preserving temporal integrity for honest evaluation
- Trains LinearRegression on historical data (first 80% of timeline)
- Evaluates on unseen future data (last 20% of timeline) using R² metric
- Shows example predictions with errors to understand model behavior

**Business value:**
Linear baseline provides interpretable performance benchmark and reveals which features have strongest linear relationships with demand. R² indicates how much demand variation the simple linear model explains.

### Challenge 4: Feature Combination Experiments

Your client asks: "Which feature types matter most - weather or time? Can we get good performance with fewer features?" Compare model performance using different feature combinations.

In [ ]:
# Your code here - compare feature combinations

print("=== FEATURE COMBINATION EXPERIMENTS ===\n")

# Define three feature sets to compare
weather_features = ['temp', 'atemp', 'humidity', 'windspeed']
time_features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month', 'season',
                 'is_rush_hour', 'is_weekend', 'is_night']
full_features = feature_columns  # All 20 features

# Train and evaluate models with each feature set
feature_sets = {
    'Weather Only': weather_features,
    'Time Only': time_features,
    'Full Feature Set': full_features
}

results = []

for name, features in feature_sets.items():
    # Train model
    X_train_subset = X_train[_____]
    X_test_subset = X_test[_____]
    
    model = LinearRegression()
    model.fit(_____, y_train)
    
    # Evaluate
    predictions = model.predict(_____)
    r2 = r2_score(y_test, _____)
    
    results.append({
        'Feature Set': name,
        'Num Features': len(_____),
        'Test R²': r2
    })
    
    print(f"{name}:")
    print(f"  Features: {len(features)}")
    print(f"  Test R²: {r2:.4f} ({r2*100:.1f}% variance explained)")
    print()

# Create comparison visualization
results_df = pd.DataFrame(results)

plt.figure(figsize=(10, 6))
bars = plt.bar(results_df['Feature Set'], results_df['Test R²'], 
               color=['skyblue', 'lightcoral', 'lightgreen'], edgecolor='black', alpha=0.8)
plt.ylabel('Test R² Score', fontsize=12)
plt.title('Feature Set Performance Comparison', fontsize=14, fontweight='bold')
plt.ylim(0, max(results_df['Test R²']) * 1.2)
plt.grid(axis='y', alpha=0.3)

# Add value labels
for bar, r2 in zip(bars, results_df['Test R²']):
    plt.text(bar.get_x() + bar.get_width()/2, r2 + 0.01, 
             f'{r2:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Business insights
best_set = results_df.loc[results_df['Test R²'].idxmax(), 'Feature Set']
print(f"\n=== INSIGHTS ===")
print(f"Best performing set: {best_set}")
print(f"Key finding: {'Time' if 'Time' in best_set else 'Weather' if 'Weather' in best_set else 'Combined'} features drive predictions")

<details>
<summary>💡 <strong>Tip</strong> (click to expand)</summary>

For each feature set, subset the training and testing data using `X_train[features]` where features is the list of column names. Train a fresh LinearRegression() model for each subset - don't reuse models! The pattern is: subset data → create new model → fit on subset training → predict on subset testing → calculate R². Store results in a list of dictionaries for easy comparison. Weather-only features test whether environmental conditions alone can predict demand (typically r² ~0.15-0.25). Time-only features test temporal patterns without weather (typically r² ~0.40-0.60). Full feature set combines both (typically r² ~0.65-0.80). The comparison reveals whether demand is primarily driven by when people ride (temporal) or what conditions encourage riding (weather). Business insight: If time features dominate, invest in temporal fleet positioning; if weather dominates, invest in weather-responsive operations; if combination is best, integrated strategy needed.
</details>

<details>
<summary>🤫 <strong>Solution</strong> (click to expand)</summary>

```python
print("=== FEATURE COMBINATION EXPERIMENTS ===\n")

# Define three feature sets
weather_features = ['temp', 'atemp', 'humidity', 'windspeed']
time_features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month', 'season',
                 'is_rush_hour', 'is_weekend', 'is_night']
full_features = feature_columns

feature_sets = {
    'Weather Only': weather_features,
    'Time Only': time_features,
    'Full Feature Set': full_features
}

results = []

for name, features in feature_sets.items():
    # Train model with feature subset
    X_train_subset = X_train[features]
    X_test_subset = X_test[features]
    
    model = LinearRegression()
    model.fit(X_train_subset, y_train)
    
    # Evaluate
    predictions = model.predict(X_test_subset)
    r2 = r2_score(y_test, predictions)
    
    results.append({
        'Feature Set': name,
        'Num Features': len(features),
        'Test R²': r2
    })
    
    print(f"{name}:")
    print(f"  Features: {len(features)}")
    print(f"  Test R²: {r2:.4f} ({r2*100:.1f}% variance explained)")
    print()

# Visualization
results_df = pd.DataFrame(results)

plt.figure(figsize=(10, 6))
bars = plt.bar(results_df['Feature Set'], results_df['Test R²'], 
               color=['skyblue', 'lightcoral', 'lightgreen'], edgecolor='black', alpha=0.8)
plt.ylabel('Test R² Score', fontsize=12)
plt.title('Feature Set Performance Comparison', fontsize=14, fontweight='bold')
plt.ylim(0, max(results_df['Test R²']) * 1.2)
plt.grid(axis='y', alpha=0.3)

for bar, r2 in zip(bars, results_df['Test R²']):
    plt.text(bar.get_x() + bar.get_width()/2, r2 + 0.01, 
             f'{r2:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Insights
best_set = results_df.loc[results_df['Test R²'].idxmax(), 'Feature Set']
time_r2 = results_df[results_df['Feature Set'] == 'Time Only']['Test R²'].values[0]
weather_r2 = results_df[results_df['Feature Set'] == 'Weather Only']['Test R²'].values[0]

print(f"=== FEATURE SET INSIGHTS ===")
print(f"✓ Best performing: {best_set}")
print(f"✓ Time features alone: {time_r2:.1%} explanatory power")
print(f"✓ Weather features alone: {weather_r2:.1%} explanatory power")
print(f"✓ Full set: {results_df[results_df['Feature Set'] == 'Full Feature Set']['Test R²'].values[0]:.1%}")
print(f"\nKey finding: {'Temporal patterns dominate' if time_r2 > weather_r2 else 'Weather conditions dominate'} bike demand")
```
</details>

---

## Step 5: Random Forest Ensemble Model

Linear regression established our baseline. Now we deploy ensemble methods to capture non-linear patterns and feature interactions. **From the lecture "Tree-Based Models"**, Random Forest combines multiple decision trees to achieve superior accuracy while reducing overfitting.

In [ ]:
# Import ensemble methods
from sklearn.ensemble import RandomForestRegressor

print("=== STEP 5: RANDOM FOREST ENSEMBLE MODEL ===\n")

# Train Random Forest (using same train-test split as Linear model)
print("--- Training Random Forest ---")
rf_model = RandomForestRegressor(
    n_estimators=100,      # 100 decision trees in the forest
    max_depth=20,          # Limit depth to control overfitting
    min_samples_split=20,  # Minimum samples required to split a node
    random_state=42,
    n_jobs=-1              # Use all CPU cores
)

rf_model.fit(X_train, y_train)

print(f"✓ Random Forest trained: {rf_model.n_estimators} trees")
print(f"✓ Tree depth limit: {rf_model.max_depth}")
print(f"✓ Training completed on {len(X_train):,} observations\n")

# Evaluate on test set
print("--- Model Evaluation ---")
test_predictions_rf = rf_model.predict(X_test)
test_r2_rf = r2_score(y_test, test_predictions_rf)

# Also evaluate on training set to check overfitting
train_predictions_rf = rf_model.predict(X_train)
train_r2_rf = r2_score(y_train, train_predictions_rf)

print(f"Training R²: {train_r2_rf:.4f} ({train_r2_rf*100:.2f}% variance explained)")
print(f"Testing R²:  {test_r2_rf:.4f} ({test_r2_rf*100:.2f}% variance explained)")
print(f"Overfit gap: {train_r2_rf - test_r2_rf:.4f}")

if (train_r2_rf - test_r2_rf) < 0.10:
    print("✓ Excellent generalization - minimal overfitting")
elif (train_r2_rf - test_r2_rf) < 0.20:
    print("✓ Good generalization - acceptable overfitting")
else:
    print("⚠ Moderate overfitting - consider regularization")

# Compare to Linear baseline
improvement = ((test_r2_rf - test_r2_linear) / test_r2_linear) * 100
print(f"\n--- Performance vs Linear Baseline ---")
print(f"Linear R²:        {test_r2_linear:.4f}")
print(f"Random Forest R²: {test_r2_rf:.4f}")
print(f"Improvement:      {improvement:+.1f}%")

if improvement > 10:
    print("✓ Significant improvement - Random Forest captures non-linear patterns")
elif improvement > 5:
    print("✓ Moderate improvement - ensemble methods add value")
else:
    print("~ Marginal improvement - linear relationships dominate")

# Show example predictions
print("\n--- Example Predictions (First 5 Test Observations) ---")
comparison_rf = pd.DataFrame({
    'Actual': y_test.iloc[:5].values,
    'Linear': test_predictions_linear[:5],
    'Random Forest': test_predictions_rf[:5],
    'RF Error': y_test.iloc[:5].values - test_predictions_rf[:5]
})
print(comparison_rf.round(1))

**What this does:**
- Trains RandomForestRegressor with 100 trees using same train-test split as Linear model
- Evaluates on test set with R² metric for direct comparison
- Checks train-test gap to assess overfitting (should be <0.20 for good generalization)
- Visualizes top 10 features with horizontal bar chart for stakeholder communication

**Business value:**
Random Forest captures non-linear relationships and feature interactions Linear models miss. Feature importance guides strategic investments - showing whether to prioritize temporal optimization or weather-responsive operations.

### Challenge 5: Deep Feature Importance Analysis

Your client asks: "Can you analyze the feature importance patterns more deeply? I want to understand cumulative importance and identify the minimal feature set that retains most predictive power." Perform comprehensive feature importance analysis.

In [ ]:
# Create feature importance DataFrame from Random Forest model
feature_importance_rf = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False).reset_index(drop=True)

# Calculate cumulative importance
feature_importance_rf['cumulative'] = feature_importance_rf['importance'].cumsum()

print("=== CUMULATIVE IMPORTANCE ANALYSIS ===\n")

# Find thresholds
for i in range(len(feature_importance_rf)):
    cumulative = feature_importance_rf.iloc[i]['cumulative']
    if i > 0:
        prev_cumulative = feature_importance_rf.iloc[i-1]['cumulative']
        if cumulative >= 0.80 and prev_cumulative < 0.80:
            print(f"Top {i+1} features capture 80% of predictive power")
        if cumulative >= 0.90 and prev_cumulative < 0.90:
            print(f"Top {i+1} features capture 90% of predictive power")

# Visualizations
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Panel 1: Individual importance
top_12 = feature_importance_rf.head(12)
colors_importance = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(top_12)))
axes[0].barh(range(len(top_12)), top_12['importance'], color=colors_importance)
axes[0].set_yticks(range(len(top_12)))
axes[0].set_yticklabels(top_12['feature'])
axes[0].invert_yaxis()
axes[0].set_xlabel('Importance Score', fontsize=11)
axes[0].set_title('Individual Feature Importance', fontsize=12, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Panel 2: Cumulative curve
axes[1].plot(range(1, len(feature_importance_rf) + 1), 
             feature_importance_rf['cumulative'], 
             'o-', linewidth=2, markersize=6, color='darkblue')
axes[1].axhline(y=0.80, color='orange', linestyle='--', linewidth=2, label='80%')
axes[1].axhline(y=0.90, color='red', linestyle='--', linewidth=2, label='90%')
axes[1].set_xlabel('Number of Top Features', fontsize=11)
axes[1].set_ylabel('Cumulative Importance', fontsize=11)
axes[1].set_title('Cumulative Feature Contribution', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 1.05)

plt.tight_layout()
plt.show()

# Test reduced model
top_features_list = feature_importance_rf.head(8)['feature'].tolist()

rf_reduced = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
rf_reduced.fit(X_train[top_features_list], y_train)
r2_reduced = r2_score(y_test, rf_reduced.predict(X_test[top_features_list]))

print(f"\n=== REDUCED FEATURE SET TEST ===")
print(f"Full model (20 features):     R² = {test_r2_rf:.4f}")
print(f"Reduced model (8 features):   R² = {r2_reduced:.4f}")
print(f"Performance retained:         {(r2_reduced/test_r2_rf)*100:.1f}%")
print(f"Complexity reduction:         {((20-8)/20)*100:.0f}% fewer features")

print(f"\n✓ Top 8 features: {', '.join(top_features_list[:4])}...")
print(f"{'✓ Recommend reduced model for production' if r2_reduced/test_r2_rf > 0.95 else '✓ Recommend full model - all features provide value'}")

<details>
<summary>💡 <strong>Tip</strong> (click to expand)</summary>

First create the feature importance DataFrame from `rf_model.feature_importances_` paired with `feature_columns`: `pd.DataFrame({'feature': feature_columns, 'importance': rf_model.feature_importances_})`. Sort by importance descending and reset the index. Cumulative importance shows how much predictive power accumulates as you add features ranked by importance. Calculate with `.cumsum()` on the sorted importance column. The cumulative curve typically shows diminishing returns: top 3 features might capture 60%, top 6 capture 80%, top 10 capture 90%. To find thresholds, iterate through cumulative values checking when they cross 0.80 and 0.90. For the reduced model test, use the top N features list: `top_features_list = feature_importance_rf.head(N)['feature'].tolist()`, then subset X_train and X_test with this list. Train a fresh Random Forest and compare its R² to the full model. Business question: Can we get 95%+ of performance with 50% fewer features? If yes, the simplified model is easier to maintain and deploy. If no, all features justify their complexity. Practical rule: If top 8 features give you 95% of performance, drop the bottom 12 for production simplicity.
</details>

<details>
<summary>🤫 <strong>Solution</strong> (click to expand)</summary>

```python
# Create feature importance DataFrame from Random Forest model
feature_importance_rf = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False).reset_index(drop=True)

# Calculate cumulative importance
feature_importance_rf['cumulative'] = feature_importance_rf['importance'].cumsum()

print("=== CUMULATIVE IMPORTANCE ANALYSIS ===\n")

# Find thresholds
for i in range(len(feature_importance_rf)):
    cumulative = feature_importance_rf.iloc[i]['cumulative']
    if i > 0:
        prev_cumulative = feature_importance_rf.iloc[i-1]['cumulative']
        if cumulative >= 0.80 and prev_cumulative < 0.80:
            print(f"Top {i+1} features capture 80% of predictive power")
        if cumulative >= 0.90 and prev_cumulative < 0.90:
            print(f"Top {i+1} features capture 90% of predictive power")

# Visualizations
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Panel 1: Individual importance
top_12 = feature_importance_rf.head(12)
colors_importance = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(top_12)))
axes[0].barh(range(len(top_12)), top_12['importance'], color=colors_importance)
axes[0].set_yticks(range(len(top_12)))
axes[0].set_yticklabels(top_12['feature'])
axes[0].invert_yaxis()
axes[0].set_xlabel('Importance Score', fontsize=11)
axes[0].set_title('Individual Feature Importance', fontsize=12, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Panel 2: Cumulative curve
axes[1].plot(range(1, len(feature_importance_rf) + 1), 
             feature_importance_rf['cumulative'], 
             'o-', linewidth=2, markersize=6, color='darkblue')
axes[1].axhline(y=0.80, color='orange', linestyle='--', linewidth=2, label='80%')
axes[1].axhline(y=0.90, color='red', linestyle='--', linewidth=2, label='90%')
axes[1].set_xlabel('Number of Top Features', fontsize=11)
axes[1].set_ylabel('Cumulative Importance', fontsize=11)
axes[1].set_title('Cumulative Feature Contribution', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 1.05)

plt.tight_layout()
plt.show()

# Test reduced model
top_features_list = feature_importance_rf.head(8)['feature'].tolist()

rf_reduced = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
rf_reduced.fit(X_train[top_features_list], y_train)
r2_reduced = r2_score(y_test, rf_reduced.predict(X_test[top_features_list]))

print(f"\n=== REDUCED FEATURE SET TEST ===")
print(f"Full model (20 features):     R² = {test_r2_rf:.4f}")
print(f"Reduced model (8 features):   R² = {r2_reduced:.4f}")
print(f"Performance retained:         {(r2_reduced/test_r2_rf)*100:.1f}%")
print(f"Complexity reduction:         {((20-8)/20)*100:.0f}% fewer features")

print(f"\n✓ Top 8 features: {', '.join(top_features_list[:4])}...")
print(f"{'✓ Recommend reduced model for production' if r2_reduced/test_r2_rf > 0.95 else '✓ Recommend full model - all features provide value'}")
```
</details>

---

## Step 6: Comprehensive Model Comparison and Deployment Decision

We've trained both a Linear Regression baseline and a Random Forest ensemble model. Now we face the critical question: **which model should we deploy to production?** This decision requires more than comparing R² scores from a single train-test split—we need statistical confidence that one model consistently outperforms the other across different time periods.

**From the lecture "Model Evaluation & Performance Assessment"**, we learned that rigorous model selection requires: (1) multiple evaluation metrics (MAE, RMSE, R²), (2) cross-validation for statistical confidence, and (3) business impact translation. We implement this framework using **TimeSeriesSplit cross-validation**, which respects temporal ordering by training on historical data and testing on future data—mimicking real-world deployment where we predict tomorrow using yesterday's patterns.

In [ ]:
# Import comprehensive evaluation metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

print("=== STEP 6: COMPREHENSIVE MODEL COMPARISON ===\n")

# Configure TimeSeriesSplit cross-validation with 5 folds
print("--- Cross-Validation Setup ---")
tscv = TimeSeriesSplit(n_splits=5)

print("Method: TimeSeriesSplit (respects temporal order)")
print("Number of folds: 5")
print("Strategy: Expanding window (each fold trains on more historical data)\n")

# Show fold structure for transparency
print("Fold structure:")
for fold, (train_idx, test_idx) in enumerate(tscv.split(X), 1):
    train_start = df_clean.iloc[train_idx[0]]['datetime'].date()
    train_end = df_clean.iloc[train_idx[-1]]['datetime'].date()
    test_start = df_clean.iloc[test_idx[0]]['datetime'].date()
    test_end = df_clean.iloc[test_idx[-1]]['datetime'].date()
    
    print(f"Fold {fold}: Train {len(train_idx):,} obs ({train_start} to {train_end}) → "
          f"Test {len(test_idx):,} obs ({test_start} to {test_end})")

# Perform cross-validation for both models
print("\n--- Running Cross-Validation for Both Models ---\n")

# Linear Regression CV
print("Linear Regression:")
linear_cv_scores = cross_val_score(LinearRegression(), X, y, cv=tscv, 
                                   scoring='r2', n_jobs=-1)
for fold, score in enumerate(linear_cv_scores, 1):
    print(f"  Fold {fold}: R² = {score:.4f}")

linear_cv_mean = linear_cv_scores.mean()
linear_cv_std = linear_cv_scores.std()
print(f"  Mean: {linear_cv_mean:.4f} ± {linear_cv_std:.4f}\n")

# Random Forest CV
print("Random Forest:")
rf_cv_scores = cross_val_score(
    RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1),
    X, y, cv=tscv, scoring='r2', n_jobs=-1
)
for fold, score in enumerate(rf_cv_scores, 1):
    print(f"  Fold {fold}: R² = {score:.4f}")

rf_cv_mean = rf_cv_scores.mean()
rf_cv_std = rf_cv_scores.std()
print(f"  Mean: {rf_cv_mean:.4f} ± {rf_cv_std:.4f}")

# Statistical comparison
print(f"\n--- Cross-Validation Statistical Comparison ---")
print(f"Linear Regression: {linear_cv_mean:.4f} ± {linear_cv_std:.4f}")
print(f"Random Forest:     {rf_cv_mean:.4f} ± {rf_cv_std:.4f}")
print(f"Difference:        {(rf_cv_mean - linear_cv_mean):+.4f}")

if rf_cv_mean > linear_cv_mean:
    improvement_pct = ((rf_cv_mean - linear_cv_mean) / linear_cv_mean) * 100
    print(f"✓ Random Forest superior: {improvement_pct:+.1f}% better mean performance")
    winner = "Random Forest"
else:
    improvement_pct = ((linear_cv_mean - rf_cv_mean) / rf_cv_mean) * 100
    print(f"✓ Linear Regression superior: {improvement_pct:+.1f}% better mean performance")
    winner = "Linear Regression"

# Calculate additional metrics on final test set
print("\n--- Test Set Performance Metrics ---")

# Use the test predictions from Step 4 and Step 5
linear_mae = mean_absolute_error(y_test, test_predictions_linear)
linear_rmse = np.sqrt(mean_squared_error(y_test, test_predictions_linear))
linear_r2 = test_r2_linear

rf_mae = mean_absolute_error(y_test, test_predictions_rf)
rf_rmse = np.sqrt(mean_squared_error(y_test, test_predictions_rf))
rf_r2 = test_r2_rf

print("\nLinear Regression Test Set:")
print(f"  MAE:  {linear_mae:.2f} bikes/hour")
print(f"  RMSE: {linear_rmse:.2f} bikes/hour")
print(f"  R²:   {linear_r2:.4f}")

print("\nRandom Forest Test Set:")
print(f"  MAE:  {rf_mae:.2f} bikes/hour")
print(f"  RMSE: {rf_rmse:.2f} bikes/hour")
print(f"  R²:   {rf_r2:.4f}")

# Create comprehensive comparison table
print(f"\n{'='*80}")
print("COMPREHENSIVE MODEL COMPARISON TABLE")
print(f"{'='*80}\n")

comparison_metrics = pd.DataFrame({
    'Metric': ['MAE (bikes/hour)', 'RMSE (bikes/hour)', 'R² Score (test)', 'R² Score (CV mean)'],
    'Linear Regression': [linear_mae, linear_rmse, linear_r2, linear_cv_mean],
    'Random Forest': [rf_mae, rf_rmse, rf_r2, rf_cv_mean],
    'Winner': [
        'RF' if rf_mae < linear_mae else 'Linear',
        'RF' if rf_rmse < linear_rmse else 'Linear',
        'RF' if rf_r2 > linear_r2 else 'Linear',
        'RF' if rf_cv_mean > linear_cv_mean else 'Linear'
    ]
})

print(comparison_metrics.to_string(index=False))

# Summary of wins
rf_wins = sum([rf_mae < linear_mae, rf_rmse < linear_rmse, rf_r2 > linear_r2, rf_cv_mean > linear_cv_mean])
print(f"\n{winner} wins {rf_wins}/4 metrics")
print(f"{'='*80}")

# Visualize comprehensive comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Comprehensive Model Comparison', fontsize=16, fontweight='bold')

# Panel 1: Cross-validation fold-by-fold performance
fold_numbers = range(1, 6)
axes[0, 0].plot(fold_numbers, linear_cv_scores, 'o-', linewidth=2.5, markersize=9,
                color='blue', label='Linear Regression', alpha=0.8)
axes[0, 0].plot(fold_numbers, rf_cv_scores, 's-', linewidth=2.5, markersize=9,
                color='green', label='Random Forest', alpha=0.8)
axes[0, 0].axhline(y=linear_cv_mean, color='blue', linestyle='--', alpha=0.5, linewidth=1.5)
axes[0, 0].axhline(y=rf_cv_mean, color='green', linestyle='--', alpha=0.5, linewidth=1.5)
axes[0, 0].set_xlabel('Fold Number', fontsize=11)
axes[0, 0].set_ylabel('R² Score', fontsize=11)
axes[0, 0].set_title('Cross-Validation Performance by Fold', fontsize=12, fontweight='bold')
axes[0, 0].set_xticks([1, 2, 3, 4, 5])  # Force integer fold numbers only
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Panel 2: Cross-validation stability (box plot)
bp = axes[0, 1].boxplot([linear_cv_scores, rf_cv_scores], 
                         tick_labels=['Linear', 'Random Forest'],
                         patch_artist=True, showmeans=True,
                         boxprops=dict(edgecolor='black', linewidth=1.5),
                         medianprops=dict(color='red', linewidth=2),
                         meanprops=dict(marker='D', markerfacecolor='gold', markersize=8))
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightgreen')
axes[0, 1].set_ylabel('R² Score', fontsize=11)
axes[0, 1].set_title('Performance Distribution (Stability)', fontsize=12, fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)

# Panel 3: Test set metrics comparison (MAE and RMSE only)
metrics_names = ['MAE\n(lower better)', 'RMSE\n(lower better)']
linear_metrics = [linear_mae, linear_rmse]
rf_metrics = [rf_mae, rf_rmse]

x = np.arange(len(metrics_names))
width = 0.35

bars1 = axes[1, 0].bar(x - width/2, linear_metrics, width, label='Linear', 
                       color='lightblue', edgecolor='black', alpha=0.8)
bars2 = axes[1, 0].bar(x + width/2, rf_metrics, width, label='Random Forest', 
                       color='lightgreen', edgecolor='black', alpha=0.8)

axes[1, 0].set_ylabel('Metric Value (bikes/hour)', fontsize=11)
axes[1, 0].set_title('Test Set Error Metrics', fontsize=12, fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(metrics_names)
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.2f}', ha='center', va='bottom', fontsize=9)

# Panel 4: R² Score Comparison (Test Set and CV Mean)
r2_categories = ['Test Set R²\n(higher better)', 'CV Mean R²\n(higher better)']
linear_r2_values = [linear_r2, linear_cv_mean]
rf_r2_values = [rf_r2, rf_cv_mean]

x_r2 = np.arange(len(r2_categories))
width_r2 = 0.35

bars_lin = axes[1, 1].bar(x_r2 - width_r2/2, linear_r2_values, width_r2, 
                          label='Linear', color='lightblue', edgecolor='black', alpha=0.8)
bars_rf = axes[1, 1].bar(x_r2 + width_r2/2, rf_r2_values, width_r2, 
                         label='Random Forest', color='lightgreen', edgecolor='black', alpha=0.8)

axes[1, 1].set_ylabel('R² Score', fontsize=11)
axes[1, 1].set_title('R² Score Comparison', fontsize=12, fontweight='bold')
axes[1, 1].set_xticks(x_r2)
axes[1, 1].set_xticklabels(r2_categories)
axes[1, 1].set_ylim(0, 1.0)
axes[1, 1].legend(loc='upper left')
axes[1, 1].grid(axis='y', alpha=0.3)

# Add value labels
for bars in [bars_lin, bars_rf]:
    for bar in bars:
        height = bar.get_height()
        axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.4f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

**What this does:**
- Uses **TimeSeriesSplit cross-validation as the primary evaluation method** (not an afterthought)
- Shows fold structure with date ranges for transparency (5 expanding windows)
- Prints fold-by-fold R² scores for both Linear Regression AND Random Forest
- Calculates CV mean and std for statistical confidence
- Computes test set metrics (MAE, RMSE, R²) for both models
- **Creates comprehensive comparison table** showing all 4 metrics (MAE, RMSE, test R², CV mean R²) with winner declared for each
- Visualizes performance with 4-panel dashboard:
  - **Panel 1**: CV fold-by-fold performance (line plot showing R² across 5 folds)
  - **Panel 2**: CV stability (box plot showing distribution and outliers)
  - **Panel 3**: Test set error metrics (MAE and RMSE in bikes/hour - same scale)
  - **Panel 4**: R² score comparison (Test Set R² and CV Mean R² on 0-1 scale)
- Shows clear summary: which model wins on how many metrics

**Business value:**
Cross-validation provides statistical confidence across multiple time periods, not just one arbitrary train-test split. The comprehensive comparison table gives stakeholders a clear, at-a-glance view of model performance across all key metrics. This rigorous approach demonstrates which model will generalize better to future unseen data.

### Challenge 6: Enhanced Model with Rolling Window Features

Your client asks: "We engineered those rolling window features in Challenge 2 - do they actually improve our models? I want to see if the extra complexity is worth it." Add the rolling features to both models and compare baseline vs enhanced performance.

In [ ]:
# Your code here - retrain models with rolling window features and compare

print("=== ENHANCED MODEL WITH ROLLING WINDOW FEATURES ===\n")

# First, create rolling window features if not already present
if 'demand_3h_avg' not in df_clean.columns:
    df_clean['demand_3h_avg'] = df_clean['count'].shift(1).rolling(window=3, min_periods=1).mean()
    df_clean['demand_24h_avg'] = df_clean['count'].shift(1).rolling(window=24, min_periods=1).mean()
    print("✓ Rolling window features created")

# Handle NaN values (whether features were just created or from Challenge 2)
df_clean['demand_3h_avg'] = df_clean['demand_3h_avg'].fillna(df_clean['count'].mean())
df_clean['demand_24h_avg'] = df_clean['demand_24h_avg'].fillna(df_clean['count'].mean())
print("✓ NaN values handled in rolling features")

# Expand feature set to include rolling features
enhanced_features = feature_columns + ['demand_3h_avg', 'demand_24h_avg']

print(f"Baseline features: {len(feature_columns)}")
print(f"Enhanced features: {len(enhanced_features)} (+2 rolling features)\n")

X_enhanced = df_clean[_____]
y_enhanced = df_clean['count']

# Cross-validation comparison
print("\n--- Cross-Validation: Baseline vs Enhanced ---")

# Linear Regression CV - Enhanced
linear_enh_cv_scores = cross_val_score(
    LinearRegression(), 
    _____, y_enhanced, 
    cv=_____, 
    scoring='r2', 
    n_jobs=-1
)
linear_enh_cv_mean = linear_enh_cv_scores._____()

# Random Forest CV - Enhanced  
rf_enh_cv_scores = cross_val_score(
    RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1),
    _____, y_enhanced,
    cv=_____,
    scoring='r2',
    n_jobs=-1
)
rf_enh_cv_mean = rf_enh_cv_scores._____()

print("\nLinear Regression CV:")
print(f"  Baseline (20 features):  {linear_cv_mean:.4f} ± {linear_cv_std:.4f}")
print(f"  Enhanced (22 features):  {linear_enh_cv_mean:.4f} ± {linear_enh_cv_scores.std():.4f}")
print(f"  Change: {(linear_enh_cv_mean - linear_cv_mean):+.4f}")

print("\nRandom Forest CV:")
print(f"  Baseline (20 features):  {rf_cv_mean:.4f} ± {rf_cv_std:.4f}")
print(f"  Enhanced (22 features):  {rf_enh_cv_mean:.4f} ± {rf_enh_cv_scores.std():.4f}")
print(f"  Change: {(rf_enh_cv_mean - rf_cv_mean):+.4f}")

# Test set evaluation (for detailed metrics and visualizations)
X_train_enh = X_enhanced.iloc[:split_index]
X_test_enh = X_enhanced.iloc[split_index:]
y_train_enh = y_enhanced.iloc[:split_index]
y_test_enh = y_enhanced.iloc[split_index:]

# Train on full training set for test predictions
linear_enhanced = LinearRegression()
linear_enhanced.fit(_____, _____)
linear_enh_pred = linear_enhanced.predict(_____)
linear_enh_r2 = r2_score(_____, _____)
linear_enh_mae = mean_absolute_error(y_test_enh, linear_enh_pred)

rf_enhanced = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
rf_enhanced.fit(_____, _____)
rf_enh_pred = rf_enhanced.predict(_____)
rf_enh_r2 = r2_score(_____, _____)
rf_enh_mae = mean_absolute_error(y_test_enh, rf_enh_pred)

print("\n--- Test Set Detailed Metrics ---")
print("\nLinear Regression:")
print(f"  Baseline:  R² = {test_r2_linear:.4f}, MAE = {linear_mae:.2f}")
print(f"  Enhanced:  R² = {linear_enh_r2:.4f}, MAE = {linear_enh_mae:.2f}")

print("\nRandom Forest:")
print(f"  Baseline:  R² = {test_r2_rf:.4f}, MAE = {rf_mae:.2f}")
print(f"  Enhanced:  R² = {rf_enh_r2:.4f}, MAE = {rf_enh_mae:.2f}")

# Feature importance for rolling features
print("\n--- ROLLING FEATURE IMPORTANCE (Enhanced RF Model) ---")
rf_enh_importance = pd.DataFrame({
    'feature': enhanced_features,
    'importance': rf_enhanced.feature_importances_
}).sort_values('importance', ascending=False)

rolling_features_imp = rf_enh_importance[rf_enh_importance['feature'].isin(
    ['demand_3h_avg', 'demand_24h_avg']
)]
print(rolling_features_imp.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Panel 1: CV Performance Comparison
models = ['Linear\nBaseline', 'Linear\nEnhanced', 'RF\nBaseline', 'RF\nEnhanced']
cv_scores = [linear_cv_mean, linear_enh_cv_mean, rf_cv_mean, rf_enh_cv_mean]
colors = ['lightblue', 'blue', 'lightgreen', 'darkgreen']

bars = axes[0].bar(models, cv_scores, color=colors, edgecolor='black', alpha=0.8)
axes[0].set_ylabel('CV Mean R² Score', fontsize=11)
axes[0].set_title('Baseline vs Enhanced (Cross-Validation)', fontsize=12, fontweight='bold')
axes[0].set_ylim(0, 1.0)
axes[0].grid(axis='y', alpha=0.3)

# Add value labels
for i, (model, score) in enumerate(zip(models, cv_scores)):
    axes[0].text(i, score + 0.02, f'{score:.4f}', ha='center', fontweight='bold', fontsize=10)

# Panel 2: Feature importance - Top 12 with rolling features highlighted
top_12_enh = rf_enh_importance.head(12)
highlight_colors = ['gold' if feat in ['demand_3h_avg', 'demand_24h_avg'] 
                    else 'steelblue' for feat in top_12_enh['feature']]

axes[1].barh(range(len(top_12_enh)), top_12_enh['importance'], 
             color=highlight_colors, edgecolor='black', alpha=0.8)
axes[1].set_yticks(range(len(top_12_enh)))
axes[1].set_yticklabels(top_12_enh['feature'])
axes[1].invert_yaxis()
axes[1].set_xlabel('Importance Score', fontsize=11)
axes[1].set_title('Top 12 Features (Rolling Window in Gold)', fontsize=12, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Decision recommendation based on CV results
print("\n" + "="*70)
max_cv_improvement = max(linear_enh_cv_mean - linear_cv_mean, rf_enh_cv_mean - rf_cv_mean)
if max_cv_improvement > 0.02:
    print("✓ RECOMMENDATION: Deploy enhanced models - rolling features provide meaningful improvement")
elif max_cv_improvement > 0.01:
    print("~ RECOMMENDATION: Rolling features provide marginal benefit - deployment decision depends on complexity tolerance")
else:
    print("✗ RECOMMENDATION: Use baseline models - rolling features add complexity without significant value")
print("="*70)

<details>
<summary>💡 <strong>Tip</strong> (click to expand)</summary>

Build the enhanced feature set: `enhanced_features = feature_columns + ['demand_3h_avg', 'demand_24h_avg']`. Create X_enhanced using this list. Following Step 6's rigorous methodology, use `cross_val_score()` with the `tscv` TimeSeriesSplit object to evaluate enhanced models: `cross_val_score(LinearRegression(), X_enhanced, y_enhanced, cv=tscv, scoring='r2')`. This gives you 5 fold scores for the enhanced model. Compare the CV means: `linear_enh_cv_scores.mean()` vs `linear_cv_mean` from Step 6. The blanks require: `X_enhanced` for the feature matrix, `tscv` for the cv parameter (reusing the TimeSeriesSplit from Step 6), and `.mean()` to get the average across folds. Also run test set evaluation for detailed metrics (MAE) and feature importance analysis - this requires the train-test split. Train fresh models: `LinearRegression()` and `RandomForestRegressor()` with identical hyperparameters. The test set blanks need: `X_train_enh, y_train_enh` for fitting, `X_test_enh` for prediction, and `y_test_enh, linear_enh_pred` for R² scoring. Fill NaN values in rolling features BEFORE creating X_enhanced to prevent "Input contains NaN" errors. Expected realistic results: Linear CV improvement 0.00-0.03, Random Forest improvement 0.00-0.02. The 3-hour rolling average captures short-term trends while the 24-hour rolling average provides daily baseline patterns - both help models understand demand trajectories beyond single lag values. Key insight: Linear models typically benefit MORE from rolling features (capturing sequential patterns they can't model), while Random Forest shows smaller gains (trees already capture patterns via existing lag features). Use CV mean improvement (not single test score) for the deployment decision - this is statistically robust. Rule of thumb: CV improvement >0.02 R² is meaningful, 0.01-0.02 is marginal, <0.01 suggests feature redundancy.
</details>

<details>
<summary>🤫 <strong>Solution</strong> (click to expand)</summary>

```python
print("=== ENHANCED MODEL WITH ROLLING WINDOW FEATURES ===\n")

# First, create rolling window features if not already present
if 'demand_3h_avg' not in df_clean.columns:
    df_clean['demand_3h_avg'] = df_clean['count'].shift(1).rolling(window=3, min_periods=1).mean()
    df_clean['demand_24h_avg'] = df_clean['count'].shift(1).rolling(window=24, min_periods=1).mean()
    print("✓ Rolling window features created")

# Handle NaN values (whether features were just created or from Challenge 2)
df_clean['demand_3h_avg'] = df_clean['demand_3h_avg'].fillna(df_clean['count'].mean())
df_clean['demand_24h_avg'] = df_clean['demand_24h_avg'].fillna(df_clean['count'].mean())
print("✓ NaN values handled in rolling features")

# Expand feature set to include rolling features
enhanced_features = feature_columns + ['demand_3h_avg', 'demand_24h_avg']

print(f"Baseline features: {len(feature_columns)}")
print(f"Enhanced features: {len(enhanced_features)} (+2 rolling features)\n")

X_enhanced = df_clean[enhanced_features]
y_enhanced = df_clean['count']

# Cross-validation comparison
print("\n--- Cross-Validation: Baseline vs Enhanced ---")

# Linear Regression CV - Enhanced
linear_enh_cv_scores = cross_val_score(
    LinearRegression(), 
    X_enhanced, y_enhanced, 
    cv=tscv, 
    scoring='r2', 
    n_jobs=-1
)
linear_enh_cv_mean = linear_enh_cv_scores.mean()

# Random Forest CV - Enhanced  
rf_enh_cv_scores = cross_val_score(
    RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1),
    X_enhanced, y_enhanced,
    cv=tscv,
    scoring='r2',
    n_jobs=-1
)
rf_enh_cv_mean = rf_enh_cv_scores.mean()

print("\nLinear Regression CV:")
print(f"  Baseline (20 features):  {linear_cv_mean:.4f} ± {linear_cv_std:.4f}")
print(f"  Enhanced (22 features):  {linear_enh_cv_mean:.4f} ± {linear_enh_cv_scores.std():.4f}")
lin_cv_change = linear_enh_cv_mean - linear_cv_mean
print(f"  Change: {lin_cv_change:+.4f} ({(lin_cv_change/linear_cv_mean)*100:+.1f}%)")

print("\nRandom Forest CV:")
print(f"  Baseline (20 features):  {rf_cv_mean:.4f} ± {rf_cv_std:.4f}")
print(f"  Enhanced (22 features):  {rf_enh_cv_mean:.4f} ± {rf_enh_cv_scores.std():.4f}")
rf_cv_change = rf_enh_cv_mean - rf_cv_mean
print(f"  Change: {rf_cv_change:+.4f} ({(rf_cv_change/rf_cv_mean)*100:+.1f}%)")

# Test set evaluation (for detailed metrics and visualizations)
X_train_enh = X_enhanced.iloc[:split_index]
X_test_enh = X_enhanced.iloc[split_index:]
y_train_enh = y_enhanced.iloc[:split_index]
y_test_enh = y_enhanced.iloc[split_index:]

print(f"\n--- Training Final Models for Test Set Evaluation ---")

# Train on full training set for test predictions
linear_enhanced = LinearRegression()
linear_enhanced.fit(X_train_enh, y_train_enh)
linear_enh_pred = linear_enhanced.predict(X_test_enh)
linear_enh_r2 = r2_score(y_test_enh, linear_enh_pred)
linear_enh_mae = mean_absolute_error(y_test_enh, linear_enh_pred)

rf_enhanced = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
rf_enhanced.fit(X_train_enh, y_train_enh)
rf_enh_pred = rf_enhanced.predict(X_test_enh)
rf_enh_r2 = r2_score(y_test_enh, rf_enh_pred)
rf_enh_mae = mean_absolute_error(y_test_enh, rf_enh_pred)

print("\n--- Test Set Detailed Metrics ---")
print("\nLinear Regression:")
print(f"  Baseline:  R² = {test_r2_linear:.4f}, MAE = {linear_mae:.2f}")
print(f"  Enhanced:  R² = {linear_enh_r2:.4f}, MAE = {linear_enh_mae:.2f}")
print(f"  Change: R² {(linear_enh_r2 - test_r2_linear):+.4f}, MAE {(linear_enh_mae - linear_mae):+.2f}")

print("\nRandom Forest:")
print(f"  Baseline:  R² = {test_r2_rf:.4f}, MAE = {rf_mae:.2f}")
print(f"  Enhanced:  R² = {rf_enh_r2:.4f}, MAE = {rf_enh_mae:.2f}")
print(f"  Change: R² {(rf_enh_r2 - test_r2_rf):+.4f}, MAE {(rf_enh_mae - rf_mae):+.2f}")

# Feature importance analysis
print("\n--- ROLLING FEATURE IMPORTANCE (Enhanced RF Model) ---")
rf_enh_importance = pd.DataFrame({
    'feature': enhanced_features,
    'importance': rf_enhanced.feature_importances_
}).sort_values('importance', ascending=False)

# Show rolling features specifically with their ranks
rolling_features_imp = rf_enh_importance[rf_enh_importance['feature'].isin(
    ['demand_3h_avg', 'demand_24h_avg']
)].copy()
rolling_features_imp['rank'] = rolling_features_imp['feature'].apply(
    lambda x: rf_enh_importance[rf_enh_importance['feature'] == x].index[0] + 1
)
print(rolling_features_imp[['feature', 'importance', 'rank']].to_string(index=False))

# Visualizations
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Panel 1: CV Performance Comparison (primary evaluation)
models = ['Linear\nBaseline', 'Linear\nEnhanced', 'RF\nBaseline', 'RF\nEnhanced']
cv_scores = [linear_cv_mean, linear_enh_cv_mean, rf_cv_mean, rf_enh_cv_mean]
colors = ['lightblue', 'blue', 'lightgreen', 'darkgreen']

bars = axes[0].bar(models, cv_scores, color=colors, edgecolor='black', alpha=0.8)
axes[0].set_ylabel('CV Mean R² Score', fontsize=11)
axes[0].set_title('Baseline vs Enhanced (Cross-Validation)', fontsize=12, fontweight='bold')
axes[0].set_ylim(0, 1.0)
axes[0].grid(axis='y', alpha=0.3)

# Add value labels
for i, (model, score) in enumerate(zip(models, cv_scores)):
    axes[0].text(i, score + 0.02, f'{score:.4f}', ha='center', fontweight='bold', fontsize=10)

# Panel 2: Feature importance with rolling features highlighted
top_12_enh = rf_enh_importance.head(12)
highlight_colors = ['gold' if feat in ['demand_3h_avg', 'demand_24h_avg'] 
                    else 'steelblue' for feat in top_12_enh['feature']]

axes[1].barh(range(len(top_12_enh)), top_12_enh['importance'], 
             color=highlight_colors, edgecolor='black', alpha=0.8)
axes[1].set_yticks(range(len(top_12_enh)))
axes[1].set_yticklabels(top_12_enh['feature'])
axes[1].invert_yaxis()
axes[1].set_xlabel('Importance Score', fontsize=11)
axes[1].set_title('Top 12 Features (Rolling Window in Gold)', fontsize=12, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

# Add importance values for rolling features
for idx, row in top_12_enh.iterrows():
    if row['feature'] in ['demand_3h_avg', 'demand_24h_avg']:
        rank = list(top_12_enh['feature']).index(row['feature'])
        axes[1].text(row['importance'], rank, f"  {row['importance']:.4f}", 
                    va='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

# Decision framework based on CV results (rigorous approach)
print("\n" + "="*70)
max_cv_improvement = max(lin_cv_change, rf_cv_change)
if max_cv_improvement > 0.02:
    print("✓ RECOMMENDATION: Deploy enhanced models")
    print(f"  Rolling features provide meaningful improvement ({max_cv_improvement:+.4f} CV R²)")
    print(f"  {'Linear benefits most' if lin_cv_change > rf_cv_change else 'Random Forest benefits most'}")
elif max_cv_improvement > 0.01:
    print("~ RECOMMENDATION: Consider enhanced models")
    print(f"  Rolling features provide marginal benefit ({max_cv_improvement:+.4f} CV R²)")
    print(f"  Deploy if operational complexity is acceptable")
else:
    print("✗ RECOMMENDATION: Use baseline models")
    print(f"  Rolling features add minimal value ({max_cv_improvement:+.4f} CV R²)")
    print(f"  Existing lag features already capture sequential patterns effectively")

# Key insights
rolling_in_top_5 = any(feat in list(rf_enh_importance.head(5)['feature']) 
                       for feat in ['demand_3h_avg', 'demand_24h_avg'])
print(f"\n✓ Rolling window features {'ARE' if rolling_in_top_5 else 'are NOT'} in top 5 importance")
print(f"✓ Linear CV improved by {(lin_cv_change/linear_cv_mean)*100:+.1f}% (linear models benefit from sequential features)")
print(f"✓ Random Forest CV improved by {(rf_cv_change/rf_cv_mean)*100:+.1f}% (trees already capture patterns)")
print(f"✓ Decision based on CV performance (statistically robust across 5 time periods)")
print("="*70)
```
</details>

---

## Summary: Complete End-to-End Machine Learning Pipeline Mastery

**What We've Accomplished**:
- **Data Quality Pipeline**: Implemented systematic validation workflows, handled missing values, verified timeline integrity, and established clean data foundation for reliable modeling
- **Feature Engineering**: Created 17 optimized features including categorical encodings, scaled weather variables, cyclical temporal features, and lag variables - transforming raw data into ML-ready inputs
- **Statistical Validation**: Applied descriptive statistics, correlation analysis, and professional visualizations (line plots, bar charts, scatter plots, box plots, heatmaps) confirming features capture real demand patterns
- **Linear Regression Baseline**: Trained interpretable baseline model achieving R² ≈ 0.80, established performance benchmark, and validated temporal train-test splitting methodology
- **Random Forest Excellence**: Deployed ensemble model with 100 trees achieving R² ≈ 0.90, analyzed feature importance revealing key demand drivers, and demonstrated non-linear pattern capture
- **Evaluation Framework**: Compared models across MAE, RMSE, and R² metrics; applied 5-fold TimeSeriesSplit cross-validation for robust estimates; translated performance to business impact; made evidence-based deployment recommendation

**Key Technical Skills Mastered**:
- **Data preparation**: Quality assessment workflows, missing data handling, timeline standardization, outlier detection (IQR method)
- **Feature engineering**: One-hot encoding, binary indicators, StandardScaler normalization, cyclical sin/cos encoding for temporal continuity, lag features with `.shift()`
- **Exploratory analysis**: `.describe()` for statistics, `.corr()` for relationships, `.groupby()` for segmentation, professional matplotlib visualizations
- **Model development**: LinearRegression and RandomForestRegressor with scikit-learn, chronological train-test splitting, proper evaluation on unseen future data
- **Performance evaluation**: MAE/RMSE/R² metric calculation, TimeSeriesSplit cross-validation, statistical confidence intervals, feature importance interpretation

Congratulations! 🥳 You've completed the full professional ML workflow from raw data to deployment-ready system. Your Capital City Bikes forecasting system demonstrates production-grade rigor across data preparation, feature engineering, model development, and evaluation - the complete skillset that distinguishes professional ML engineers from basic model builders. You're now ready to tackle real-world transportation forecasting challenges with confidence, systematic methodology, and stakeholder-ready communication!